<a href="https://colab.research.google.com/github/Hanbin-git/dacon_new_drug/blob/main/20250626(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import zipfile

zip_path = '/content/drive/MyDrive/data.zip'
extract_path = '/content/project_data'  # 원하는 경로

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [2]:
import pandas as pd

path = '/content/project_data/'  # 압축 해제 경로
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

print(train.shape, test.shape, submission.shape)


(1681, 3) (100, 2) (100, 2)


In [3]:
# RDKit 설치 (Colab에서 가능)
!pip uninstall -y rdkit-pypi
!pip install rdkit-pypi==2022.9.5 optuna xgboost



Found existing installation: rdkit-pypi 2022.9.5
Uninstalling rdkit-pypi-2022.9.5:
  Successfully uninstalled rdkit-pypi-2022.9.5
  Using cached rdkit_pypi-2022.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.9 kB)
Using cached rdkit_pypi-2022.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (29.4 MB)


In [ ]:
!pip install numpy==1.23.5
import os
os.kill(os.getpid(), 9)  # 런타임 재시작 (필수)


In [1]:
!pip install rdkit-pypi catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 5.6 MB/s eta 0:00:00


In [1]:
!pip uninstall xgboost -y
!pip install xgboost==1.6.0

Found existing installation: xgboost 1.6.0
Uninstalling xgboost-1.6.0:
  Successfully uninstalled xgboost-1.6.0
  Using cached xgboost-1.6.0-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
Using cached xgboost-1.6.0-py3-none-manylinux2014_x86_64.whl (193.7 MB)


In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, MACCSkeys, AllChem
from sklearn.model_selection import GroupKFold, StratifiedGroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
# from sklearn.linear_model import Ridge # Part 2에서 사용
# from sklearn.metrics import mean_absolute_error # Part 2에서 사용
# from sklearn.ensemble import StackingRegressor # Part 2에서 사용
from xgboost import XGBRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import optuna
import warnings
# import matplotlib.pyplot as plt # Part 2에서 사용
# import seaborn as sns # Part 2에서 사용
import time

# 경고 메시지 무시 설정
warnings.filterwarnings("ignore")

# --- scikit-learn 메타데이터 라우팅 활성화 (필수) ---
# 이 부분은 StackingRegressor의 groups 오류 해결에 필요하므로 Part 1에 포함시켜도 무방합니다.
# 만약 Part 1에서 StackingRegressor를 임포트하지 않으면 에러가 나지 않겠지만,
# 일관성을 위해 미리 활성화해두는 것이 좋습니다.
from sklearn import set_config
set_config(enable_metadata_routing=True)
# ----------------------------------------

print("--- Boost up AI 2025: 신약개발 경진대회 예측 파이프라인 - Part 1 시작 ---")
print(f"현재 시간: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())}")

# --- 데이터 경로 설정 및 로드 ---
path = '/content/project_data/'
try:
    train = pd.read_csv(path + 'train.csv')
    test = pd.read_csv(path + 'test.csv')
    submission = pd.read_csv(path + 'sample_submission.csv')
    print("데이터 로드 성공.")
except FileNotFoundError:
    print(f"오류: '{path}' 경로에 데이터 파일이 없습니다. Google Colab 좌측 파일 아이콘을 통해 데이터를 업로드하거나 경로를 확인해주세요.")
    print("예시 데이터 생성 중...")
    n_train = 1681
    n_test = 100
    train = pd.DataFrame({
        'ID': range(1, n_train + 1),
        'Canonical_Smiles': ['CCO', 'CCC', 'C(C(=O)O)N', 'C1CCCCC1', 'CC(=O)Oc1ccccc1C(=O)O'] * (n_train // 5) + ['CCO'] * (n_train % 5),
        'Inhibition': np.random.rand(n_train) * 100
    })
    test = pd.DataFrame({
        'ID': range(n_train + 1, n_train + n_test + 1),
        'Canonical_Smiles': ['CN', 'CO', 'CC(C)C', 'c1ccccc1', 'NCCCN'] * (n_test // 5) + ['CN'] * (n_test % 5)
    })
    submission = pd.DataFrame({'ID': test['ID'], 'Inhibition': 0.0})
    print("예시 데이터 생성이 완료되었습니다. 실제 대회 데이터를 사용해주세요.")

# --- 1. 'Canonical_Smiles' 컬럼 사용 확인 ---
print("\n--- 1. 'Canonical_Smiles' 컬럼 사용 확인 ---")
train_smiles_col = 'Canonical_Smiles'
test_smiles_col = 'Canonical_Smiles'

invalid_train_smiles_count = train[train_smiles_col].isnull().sum()
invalid_test_smiles_count = test[test_smiles_col].isnull().sum()

if invalid_train_smiles_count > 0:
    print(f"경고: Train 데이터의 '{train_smiles_col}'에 유효하지 않은 SMILES({invalid_train_smiles_count}개)가 있습니다. 피처 생성 시 0으로 처리됩니다.")
if invalid_test_smiles_count > 0:
    print(f"경고: Test 데이터의 '{test_smiles_col}'에 유효하지 않은 SMILES({invalid_test_smiles_count}개)가 있습니다. 피처 생성 시 0으로 처리됩니다.")
if invalid_train_smiles_count == 0 and invalid_test_smiles_count == 0:
    print(f"'{train_smiles_col}' 및 '{test_smiles_col}' 컬럼에 유효하지 않은 (None) SMILES는 없습니다.")


# --- 2. RDKit 기반 피처 생성 ---
all_descriptor_functions = [desc_func for desc_name, desc_func in Descriptors._descList]
num_descriptors = len(all_descriptor_functions)
morgan_nbits = 2048
maccs_len = 167

def get_features(smiles):
    mol = Chem.MolFromSmiles(str(smiles))
    if mol is None:
        return [0] * (num_descriptors + morgan_nbits + maccs_len)

    desc_values = []
    for desc_func in all_descriptor_functions:
        try:
            val = desc_func(mol)
            if np.isnan(val) or np.isinf(val):
                desc_values.append(0)
            else:
                desc_values.append(val)
        except Exception:
            desc_values.append(0)

    try:
        morgan = list(AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=morgan_nbits))
    except Exception:
        morgan = [0] * morgan_nbits

    try:
        maccs = list(MACCSkeys.GenMACCSKeys(mol))
    except Exception:
        maccs = [0] * maccs_len

    return desc_values + morgan + maccs

print("\n--- 2. 피처 생성 중 (약 1~2분 소요 예상) ---")
X = np.array([get_features(s) for s in train[train_smiles_col]])
X_test = np.array([get_features(s) for s in test[test_smiles_col]])
y = train['Inhibition'].values
groups = train[train_smiles_col].values # groups도 여기에 정의

print(f"생성된 Train 피처 차원: {X.shape}")
print(f"생성된 Test 피처 차원: {X_test.shape}")

# --- 3. 스케일링 및 차원 축소 (PCA) ---
print("\n--- 3. 스케일링 및 PCA 차원 축소 ---")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

X_scaled = np.nan_to_num(X_scaled, nan=0.0, posinf=0.0, neginf=0.0)
X_test_scaled = np.nan_to_num(X_test_scaled, nan=0.0, posinf=0.0, neginf=0.0)

pca = PCA(n_components=300, random_state=42)
X_pca = pca.fit_transform(X_scaled)
X_test_pca = pca.transform(X_test_scaled)

print(f"PCA 후 Train 피처 차원: {X_pca.shape}")
print(f"PCA 후 Test 피처 차원: {X_test_pca.shape}")
print(f"PCA 누적 설명 분산 (300개 컴포넌트): {np.sum(pca.explained_variance_ratio_):.4f}")

# --- 4. StratifiedGroupKFold를 위한 타겟 값 Binning ---
print("\n--- 4. StratifiedGroupKFold를 위한 Inhibition 값 Binning (튜닝 단계용) ---")
bins = pd.qcut(y, q=5, labels=False, duplicates='drop')
print(f"Binning 결과 (상위 5개): {bins[:5]}")
print(f"각 Bin별 데이터 개수:\n{pd.Series(bins).value_counts().sort_index()}")


# --- 5. Optuna를 이용한 Base 모델 하이퍼파라미터 튜닝 ---
def tune_model(X_data, y_data, groups_data, bins_data, model_type, n_trials=50, cv_folds=5, early_stopping_rounds=50):
    def objective(trial):
        if model_type == 'lgbm':
            params = {
                'objective': 'mae',
                'metric': 'mae',
                'n_estimators': trial.suggest_int("n_estimators", 100, 1000),
                'max_depth': trial.suggest_int("max_depth", 3, 15),
                'learning_rate': trial.suggest_float("learning_rate", 0.01, 0.2),
                'reg_lambda': trial.suggest_float("reg_lambda", 1e-3, 10.0),
                'reg_alpha': trial.suggest_float("reg_alpha", 1e-3, 10.0),
                'colsample_bytree': trial.suggest_float("colsample_bytree", 0.6, 1.0),
                'subsample': trial.suggest_float("subsample", 0.6, 1.0),
                'n_jobs': -1,
                'random_state': 42,
                'verbose': -1,
            }
            model = LGBMRegressor(**params)
        elif model_type == 'xgb':
            params = {
                'objective': 'reg:squarederror',
                'eval_metric': 'mae',
                'n_estimators': trial.suggest_int("n_estimators", 100, 1000),
                'max_depth': trial.suggest_int("max_depth", 3, 15),
                'learning_rate': trial.suggest_float("learning_rate", 0.01, 0.2),
                'subsample': trial.suggest_float("subsample", 0.6, 1.0),
                'colsample_bytree': trial.suggest_float("colsample_bytree", 0.6, 1.0),
                'gamma': trial.suggest_float("gamma", 1e-3, 10.0),
                'lambda': trial.suggest_float("lambda", 1e-3, 10.0),
                'alpha': trial.suggest_float("alpha", 1e-3, 10.0),
                'n_jobs': -1,
                'random_state': 42,
                'tree_method': 'hist',
            }
            model = XGBRegressor(**params)
        elif model_type == 'cat':
            params = {
                'loss_function': 'MAE',
                'iterations': trial.suggest_int("iterations", 100, 1000),
                'depth': trial.suggest_int("depth", 3, 10),
                'learning_rate': trial.suggest_float("learning_rate", 0.01, 0.2),
                'l2_leaf_reg': trial.suggest_float("l2_leaf_reg", 1e-3, 10.0),
                'border_count': trial.suggest_int("border_count", 32, 255),
                'verbose': 0,
                'random_seed': 42,
            }
            model = CatBoostRegressor(**params)
        else:
            raise ValueError(f"알 수 없는 모델 타입: {model_type}")

        sgkf = StratifiedGroupKFold(n_splits=cv_folds, shuffle=True, random_state=42)
        scores = []
        for train_idx, val_idx in sgkf.split(X_data, bins_data, groups_data):
            X_train, X_val = X_data[train_idx], X_data[val_idx]
            y_train, y_val = y_data[train_idx], y_data[val_idx]

            # 조기 종료 (Early Stopping) 반영
            if model_type == 'lgbm':
                model.fit(X_train, y_train, eval_set=[(X_val, y_val)],
                          callbacks=[lgb.early_stopping(stopping_rounds=early_stopping_rounds, verbose=False)])
            elif model_type == 'xgb':
                model.fit(X_train, y_train, eval_set=[(X_val, y_val)],
                          early_stopping_rounds=early_stopping_rounds, verbose=False)
            elif model_type == 'cat':
                model.fit(X_train, y_train, eval_set=[(X_val, y_val)],
                          early_stopping_rounds=early_stopping_rounds, verbose=False)

            pred = model.predict(X_val)
            scores.append(mean_absolute_error(y_val, pred))
        return np.mean(scores)

    print(f"\n--- 5. Optuna 튜닝 시작: {model_type} (n_trials={n_trials}) ---")
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42),
                                 study_name=f"{model_type}_tuning")
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
    print(f"최적 MAE: {study.best_value:.4f}")
    print(f"최적 파라미터: {study.best_params}")
    return study.best_params

# Optuna 튜닝 실행 (n_trials를 줄여서 빠르게 테스트할 수도 있습니다.)
# 이 파라미터들은 Part 2에서 사용되므로, 변수에 저장해둡니다.
best_lgbm_params = tune_model(X_pca, y, groups, bins, 'lgbm', n_trials=10, early_stopping_rounds=50)
best_xgb_params = tune_model(X_pca, y, groups, bins, 'xgb', n_trials=10, early_stopping_rounds=50)
best_cat_params = tune_model(X_pca, y, groups, bins, 'cat', n_trials=10, early_stopping_rounds=50)

print("\n--- Boost up AI 2025: 신약개발 경진대회 예측 파이프라인 - Part 1 완료 ---")
print("이제 Part 2를 실행할 준비가 되었습니다.")

--- Boost up AI 2025: 신약개발 경진대회 예측 파이프라인 - Part 1 시작 ---
현재 시간: 2025-06-26 04:14:23
데이터 로드 성공.

--- 1. 'Canonical_Smiles' 컬럼 사용 확인 ---
'Canonical_Smiles' 및 'Canonical_Smiles' 컬럼에 유효하지 않은 (None) SMILES는 없습니다.

--- 2. 피처 생성 중 (약 1~2분 소요 예상) ---
생성된 Train 피처 차원: (1681, 2423)
생성된 Test 피처 차원: (100, 2423)

--- 3. 스케일링 및 PCA 차원 축소 ---


[I 2025-06-26 04:15:08,883] A new study created in memory with name: lgbm_tuning


PCA 후 Train 피처 차원: (1681, 300)
PCA 후 Test 피처 차원: (100, 300)
PCA 누적 설명 분산 (300개 컴포넌트): 0.5438

--- 4. StratifiedGroupKFold를 위한 Inhibition 값 Binning (튜닝 단계용) ---
Binning 결과 (상위 5개): [1 0 0 4 1]
각 Bin별 데이터 개수:
0    337
1    336
2    338
3    334
4    336
Name: count, dtype: int64

--- 5. Optuna 튜닝 시작: lgbm (n_trials=10) ---


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2025-06-26 04:15:36,774] Trial 0 finished with value: 20.671573967514778 and parameters: {'n_estimators': 437, 'max_depth': 15, 'learning_rate': 0.14907884894416698, 'reg_lambda': 5.986986183486169, 'reg_alpha': 1.5610303857839227, 'colsample_bytree': 0.662397808134481, 'subsample': 0.6232334448672797}. Best is trial 0 with value: 20.671573967514778.
[I 2025-06-26 04:16:05,441] Trial 1 finished with value: 20.52650972989128 and parameters: {'n_estimators': 880, 'max_depth': 10, 'learning_rate': 0.14453378978124864, 'reg_lambda': 0.20682435846372868, 'reg_alpha': 9.699128611767781, 'colsample_bytree': 0.9329770563201687, 'subsample': 0.6849356442713105}. Best is trial 1 with value: 20.52650972989128.
[I 2025-06-26 04:16:21,432] Trial 2 finished with value: 20.465821896772013 and parameters: {'n_estimators': 263, 'max_depth': 5, 'learning_rate': 0.06780602616231217, 'reg_lambda': 5.2480395598907466, 'reg_alpha': 4.320018241402516, 'colsample_bytree': 0.7164916560792167, 'subsample': 0

[I 2025-06-26 04:18:18,045] A new study created in memory with name: xgb_tuning


[I 2025-06-26 04:18:18,039] Trial 9 finished with value: 20.52347950254516 and parameters: {'n_estimators': 421, 'max_depth': 6, 'learning_rate': 0.1131122558000672, 'reg_lambda': 1.4101013255226518, 'reg_alpha': 8.022167610559643, 'colsample_bytree': 0.6298202574719083, 'subsample': 0.9947547746402069}. Best is trial 5 with value: 20.383560377190015.
최적 MAE: 20.3836
최적 파라미터: {'n_estimators': 828, 'max_depth': 6, 'learning_rate': 0.028557701661212936, 'reg_lambda': 6.8426460320950575, 'reg_alpha': 4.402084784902274, 'colsample_bytree': 0.6488152939379115, 'subsample': 0.798070764044508}

--- 5. Optuna 튜닝 시작: xgb (n_trials=10) ---


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2025-06-26 04:20:25,069] Trial 0 finished with value: 20.718488801739596 and parameters: {'n_estimators': 437, 'max_depth': 15, 'learning_rate': 0.14907884894416698, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'gamma': 1.5607892088416901, 'lambda': 0.5817780380698264, 'alpha': 8.661895281603577}. Best is trial 0 with value: 20.718488801739596.


In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GroupKFold # Stacking CV에 사용
from sklearn.linear_model import Ridge # Stacking Regressor의 final_estimator
from sklearn.metrics import mean_absolute_error # 평가 지표
from sklearn.ensemble import StackingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# 중요: Part 1에서 set_config(enable_metadata_routing=True)가 실행되었음을 전제로 합니다.
# 만약 Colab 세션을 다시 시작했다면, set_config 라인을 이 Part 2 코드에도 추가해야 합니다.
# from sklearn import set_config
# set_config(enable_metadata_routing=True) # <-- 필요시 이 주석을 해제하고 실행

print("--- Boost up AI 2025: 신약개발 경진대회 예측 파이프라인 - Part 2 시작 ---")
print(f"현재 시간: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())}")

# --- Stacking Regressor 모델 정의 및 학습 ---
print("\n--- 6. Stacking Regressor 모델 정의 및 학습 ---")

# Part 1에서 튜닝된 최적 파라미터를 사용하여 base_models를 정의합니다.
# 이 변수들은 Part 1이 성공적으로 실행된 후 메모리에 남아 있어야 합니다.
# (예: best_lgbm_params, best_xgb_params, best_cat_params)
base_models = [
    ('lgb', LGBMRegressor(**best_lgbm_params)),
    ('xgb', XGBRegressor(**best_xgb_params)),
    ('cat', CatBoostRegressor(**best_cat_params, verbose=0))
]

stack_model = StackingRegressor(
    estimators=base_models,
    final_estimator=Ridge(random_state=42),
    cv=GroupKFold(n_splits=5), # Stacking 내부 CV에 GroupKFold 사용
    n_jobs=-1 # 가능한 모든 코어 사용
)

print("Stacking 모델 학습 시작...")
# X_pca, y, groups 변수들은 Part 1이 실행된 후 메모리에 남아 있어야 합니다.
stack_model.fit(X_pca, y, groups=groups)
print("Stacking 모델 학습 완료.")

# --- 7. 예측 및 후처리 ---
print("\n--- 7. 예측 및 후처리 ---")
# X_test_pca 변수는 Part 1이 실행된 후 메모리에 남아 있어야 합니다.
preds = stack_model.predict(X_test_pca)
preds = np.clip(preds, 0, 100)

print(f"최종 예측값 평균: {np.mean(preds):.2f}")
print(f"최종 예측값 표준편차: {np.std(preds):.2f}")
print(f"최종 예측값 최소: {np.min(preds):.2f}")
print(f"최종 예측값 최대: {np.max(preds):.2f}")

# --- 8. 성능 검증 및 시각화 (결과 미리 확인) ---
print("\n--- 8. 성능 검증 및 시각화 ---")

print("Stacking 모델 교차 검증 MAE 계산 중...")
cv_scores = []
gkf_final_eval = GroupKFold(n_splits=5)

# Stacking 모델의 OOF 예측을 얻기 위한 수동 교차 검증 루프 (시간 소요)
# 이 부분은 stack_model.fit() 과정에서 내부적으로 OOF 예측을 생성하여
# 메타 모델 학습에 사용하므로, 별도로 계산하는 것은 최종 평가/확인 용도입니다.
for fold, (train_idx, val_idx) in enumerate(gkf_final_eval.split(X_pca, groups=groups)):
    print(f"    - 폴드 {fold+1}/{gkf_final_eval.n_splits} 평가 중...")

    # 각 폴드마다 새로운 StackingRegressor 인스턴스 생성 및 학습
    fold_stack_model = StackingRegressor(
        estimators=base_models,
        final_estimator=Ridge(random_state=42),
        cv=GroupKFold(n_splits=5), # 내부 CV
        n_jobs=-1
    )
    # StackingRegressor 학습 시 groups를 전달
    fold_stack_model.fit(X_pca[train_idx], y[train_idx], groups=groups[train_idx])
    val_preds = fold_stack_model.predict(X_pca[val_idx])

    val_preds = np.clip(val_preds, 0, 100)

    mae = mean_absolute_error(y[val_idx], val_preds)
    cv_scores.append(mae)
    print(f"      MAE: {mae:.4f}")

print(f"\nStacking 모델 최종 교차 검증 MAE: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")

print("예측값 분포 및 잔차 분석 시각화 생성 중...")

# 훈련 데이터에 대한 예측 (시각화용)
train_preds = stack_model.predict(X_pca)
train_preds = np.clip(train_preds, 0, 100)

plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
sns.histplot(y, color='blue', label='실제 저해율 (%) (훈련)', kde=True, stat='density', linewidth=0, alpha=0.6)
sns.histplot(train_preds, color='orange', label='예측 저해율 (%) (훈련)', kde=True, stat='density', linewidth=0, alpha=0.5)
sns.histplot(preds, color='green', label='예측 저해율 (%) (테스트)', kde=True, stat='density', linewidth=0, alpha=0.6) # alpha 조정
plt.title('저해율 (%) 값 분포')
plt.xlabel('저해율 (%)')
plt.ylabel('밀도')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.subplot(1, 3, 2)
residuals = y - train_preds
sns.histplot(residuals, kde=True, color='purple', stat='density', linewidth=0)
plt.title('훈련 데이터 잔차 (실제값 - 예측값) 분포')
plt.xlabel('잔차')
plt.ylabel('밀도')
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.subplot(1, 3, 3)
plt.scatter(y, residuals, alpha=0.3, color='teal')
plt.axhline(0, color='red', linestyle='--', label='잔차 0')
plt.title('훈련 데이터 잔차 vs. 실제 저해율 (%)')
plt.xlabel('실제 저해율 (%)')
plt.ylabel('잔차 (실제값 - 예측값)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# --- 9. 제출 파일 생성 ---
print("\n--- 9. 제출 파일 생성 ---")
# submission 변수는 Part 1이 실행된 후 메모리에 남아 있어야 합니다.
submission['Inhibition'] = preds
submission.to_csv('/content/submission_optuna_stacking_enhanced.csv', index=False)
print("submission_optuna_stacking_enhanced.csv 파일이 성공적으로 생성되었습니다.")
print(f"최종 예측 결과 파일: /content/submission_optuna_stacking_enhanced.csv")
print(f"파이프라인 종료 시간: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())}")
print("\n--- Boost up AI 2025: 신약개발 경진대회 예측 파이프라인 완료 ---")